In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/results/tokenizer/tokenizer.json
/kaggle/input/results/tokenizer/tokenizer_config.json
/kaggle/input/results/tokenizer/special_tokens_map.json
/kaggle/input/results/finetuned_model/config.json
/kaggle/input/results/finetuned_model/model.safetensors
/kaggle/input/daigt-one-place-all-data/concatenated.csv
/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv


In [2]:
!pip install transformers torch

In [3]:
!pip install accelerate -U

In [4]:
import pandas as pd
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
data = pd.read_csv("/kaggle/input/daigt-one-place-all-data/concatenated.csv", sep=',')

In [5]:
data

,id,prompt_id,text,generated,model,kaggle_repo
0,d429f032,0,Advantages of Limiting Car Usage \n\nLimiting ...,1,gpt-3.5-turbo,1
1,1ce279be,0,Advantages of Limiting Car Usage\n\nLimiting c...,1,gpt-3.5-turbo,1
2,c9595213,0,Limiting car usage has numerous advantages tha...,1,gpt-3.5-turbo,1
3,f2266d87,0,The passages provided discuss the advantages o...,1,gpt-3.5-turbo,1
4,eeace4bd,0,Title: The Advantages of Limiting Car Usage\n\...,1,gpt-3.5-turbo,1
...,...,...,...,...,...,...
54686,df8bf6e6-eca4-417c-bce2-b3b90d782d93,-1,I believe using cellphones in class for educat...,0,human,9
54687,502c11ac-7d0a-4b11-8572-5f7317364bc7,-1,"Working alone, students do not have to argue w...",0,human,9
54688,993f33f3-0cfa-4b5a-b382-bdf7f82e2666,-1,"""A problem is a chance for you to do your best...",0,human,9
54689,0f9f0196-4828-4122-b0cb-aea2373066fb,-1,Many people disagree with Albert Schweitzer's ...,0,human,9


In [6]:
X = data['text']
Y = data['generated']

from sklearn.model_selection import train_test_split

# Split the data into training and validation sets with a 80-20 ratio
train, val, train_labels, eval_labels = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [8]:
train_labels.reset_index(drop=True, inplace=True)
eval_labels.reset_index(drop=True, inplace=True)

In [9]:
train.head()

0    Dear Principal Johnson, \n\nI am writing to ex...
1    To the principal,\n\nAs a student, I think the...
2    Being a good person and a good citizen is esse...
3    Texting and Driving\n\nCellphone is the most u...
4    Twenty-five years ago, a strange image was pro...
Name: text, dtype: object

In [10]:
train.shape

(43752,)

In [11]:
val.head()

0    Meditation and mindfulness practices in school...
1    Studying Venus Remains Worth Pursuing Despite ...
2    Gun control is a big issue in a lot of places....
3    Viideo games are the best! But some peoples sa...
4    As students, we all know how much homework we ...
Name: text, dtype: object

In [12]:
val.shape

(10939,)

In [13]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import torch

In [14]:
#Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/results/tokenizer")
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/results/finetuned_model")

In [15]:
max_length = 64
#Tokenize the datasets with the specified maximum length
# train_encodings = tokenizer([x for x in train], truncation=True, padding=True, max_length=max_length, return_tensors='pt')
# eval_encodings = tokenizer([x for x in val], truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer([x for x in test], truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [16]:
test_labels = [0]*len(test)

In [17]:
from torch.utils.data import DataLoader, Dataset
# Create PyTorch datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# train_dataset = Dataset(train_encodings, train_labels)
# eval_dataset = Dataset(eval_encodings, eval_labels)
test_dataset = Dataset(test_encodings, test_labels)

# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# eval_dataloader = DataLoader(eval_dataset, batch_size=4, shuffle=True)

### Manual Training Loop

In [18]:
# from tqdm import tqdm
# import torch
# from sklearn.metrics import accuracy_score

# # Assuming you have the necessary imports and datasets already defined

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./finetuned_model",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=1,
#     save_steps=500,
#     save_total_limit=2,
#     learning_rate=2e-5,
#     report_to=[]
# )

In [19]:
# # Define the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Move the model to the device
# model.to(device)

In [20]:
# # Define optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

# # Manual training loop
# for epoch in range(training_args.num_train_epochs):
#     model.train()
#     total_loss = 0.0

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch}"):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         loss.backward()
#         optimizer.step()

#     avg_train_loss = total_loss / len(train_dataloader)

#     # Evaluation after each epoch
#     model.eval()
#     all_preds = []
#     all_labels = []

#     with torch.no_grad():
#         for batch in tqdm(eval_dataloader, desc=f"Evaluating"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)

#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             logits = outputs.logits

#             preds = torch.argmax(logits, dim=1).cpu().numpy()
#             all_preds.extend(preds)
#             all_labels.extend(labels.cpu().numpy())

#     accuracy = accuracy_score(all_labels, all_preds)
#     print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}, Accuracy: {accuracy}")

In [21]:
# # Save the fine-tuned model
# model.save_pretrained('./finetuned_model')
# tokenizer.save_pretrained('./tokenizer')

In [22]:
# from sklearn.metrics import accuracy_score

# # Custom callback function to calculate and print accuracy during evaluation
# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return {"accuracy": accuracy_score(p.label_ids, preds)}

In [23]:
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./finetuned_model",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=1,
#     save_steps=500,
#     save_total_limit=2,
#     learning_rate=2e-5,
#     report_to=[]
# )

In [24]:
# # Define Trainer with custom callback
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01),
#                PrintTableCallback(early_stopping_patience=3, early_stopping_threshold=0.01),
#                LogMetricsCallback()],
#     compute_metrics=compute_metrics  # Specify the custom metric function
# )

In [25]:
# # Fine-tune the model
# trainer.train()

In [26]:
# # Save the fine-tuned model
# model.save_pretrained("./finetuned_model")

In [27]:
# # Inference on the validation set
# validation_predictions = trainer.predict(eval_dataset)
# validation_probabilities = torch.nn.functional.softmax(torch.tensor(validation_predictions.predictions), dim=1)[:, 1].numpy()

# # Calculate accuracy on validation set using y_validation list
# predicted_labels = validation_probabilities > 0.5
# accuracy = accuracy_score(eval_labels, predicted_labels)

# print(f"Accuracy on the validation set: {accuracy}")

In [28]:
# import os
# import shutil
# from zipfile import ZipFile

# # Set the output directory in Kaggle
# output_dir = '/kaggle/working/'

# # Set the model directory name
# model_directory = 'finetuned_model'

# # Set the destination path for the zip file
# zip_file_path = f'finetuned_model.zip'

# # Zip the model directory
# shutil.make_archive(zip_file_path, 'zip', os.path.join(output_dir, model_directory))

# # Move the zip file to the Kaggle output directory
# shutil.move(zip_file_path + '.zip', '/kaggle/working/')

In [29]:
# # Load the fine-tuned model
# model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/model-finetuned/finetuned_model")

### Inference on test data

In [30]:
# Create PyTorch dataloaders
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
# eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=32, shuffle=False)

In [31]:
# Put the model in evaluation mode
model.eval()

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [32]:
test_dataloader

In [33]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model.to(device)

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [34]:
import torch.nn.functional as F
import numpy as np

all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to device
        batch = {key: val.to(device) for key, val in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        
        # Get predicted labels
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        
        # Collect predictions, true labels, and probabilities
        all_preds.extend(preds)
        all_labels.extend(batch['labels'].cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        
#         print(f'predictions length= {len(all_preds)}\n probs length = {len(all_probs)}')

In [35]:
sub['generated'] = [x[1] for x in all_probs]
sub.to_csv('submission.csv', index=False)
sub

,id,generated
0,0000aaaa,0.001273
1,1111bbbb,0.012865
2,2222cccc,0.050954
